## LangGraph Open Deep Research - Supervisor-Researcher Architecture

In this notebook, we'll explore the **supervisor-researcher delegation architecture** for conducting deep research with LangGraph.

You can visit this repository to see the original application: [Open Deep Research](https://github.com/langchain-ai/open_deep_research)

Let's jump in!

## What We're Building

This implementation uses a **hierarchical delegation pattern** where:

1. **User Clarification** - Optionally asks clarifying questions to understand the research scope
2. **Research Brief Generation** - Transforms user messages into a structured research brief
3. **Supervisor** - A lead researcher that analyzes the brief and delegates research tasks
4. **Parallel Researchers** - Multiple sub-agents that conduct focused research simultaneously
5. **Research Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings are combined into a comprehensive report

![Architecture Diagram](https://i.imgur.com/Q8HEZn0.png)

This differs from a section-based approach by allowing dynamic task decomposition based on the research question, rather than predefined sections.

---

# 🤝 Breakout Room #1
## Deep Research Foundations

In this breakout room, we'll understand the architecture and components of the Open Deep Research system.

## Task 1: Dependencies

You'll need API keys for Anthropic (for the LLM) and Tavily (for web search). We'll configure the system to use Anthropic's Claude Sonnet 4 exclusively.

In [1]:
import os
import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Enter your Anthropic API key: ")
os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

In [1]:
import os
# Set API Keys
os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
# Tavily API Key for web search
os.environ["TAVILY_API_KEY"] = os.environ.get("TAVILY_API_KEY")

## Task 2: State Definitions

The state structure is hierarchical with three levels:

### Agent State (Top Level)
Contains the overall conversation messages, research brief, accumulated notes, and final report.

### Supervisor State (Middle Level)
Manages the research supervisor's messages, research iterations, and coordinating parallel researchers.

### Researcher State (Bottom Level)
Each individual researcher has their own message history, tool call iterations, and research findings.

We also have structured outputs for tool calling:
- **ConductResearch** - Tool for supervisor to delegate research to a sub-agent
- **ResearchComplete** - Tool to signal research phase is done
- **ClarifyWithUser** - Structured output for asking clarifying questions
- **ResearchQuestion** - Structured output for the research brief

Let's import these from our library: [`open_deep_library/state.py`](open_deep_library/state.py)

In [2]:
# Import state definitions from the library
from open_deep_library.state import (
    # Main workflow states
    AgentState,           # Lines 65-72: Top-level agent state with messages, research_brief, notes, final_report
    AgentInputState,      # Lines 62-63: Input state is just messages
    
    # Supervisor states
    SupervisorState,      # Lines 74-81: Supervisor manages research delegation and iterations
    
    # Researcher states
    ResearcherState,      # Lines 83-90: Individual researcher with messages and tool iterations
    ResearcherOutputState, # Lines 92-96: Output from researcher (compressed research + raw notes)
    
    # Structured outputs for tool calling
    ConductResearch,      # Lines 15-19: Tool for delegating research to sub-agents
    ResearchComplete,     # Lines 21-22: Tool to signal research completion
    ClarifyWithUser,      # Lines 30-41: Structured output for user clarification
    ResearchQuestion,     # Lines 43-48: Structured output for research brief
)

## Task 3: Utility Functions and Tools

The system uses several key utilities:

### Search Tools
- **tavily_search** - Async web search with automatic summarization to stay within token limits
- Supports Anthropic native web search and Tavily API

### Reflection Tools
- **think_tool** - Allows researchers to reflect on their progress and plan next steps (ReAct pattern)

### Helper Utilities
- **get_all_tools** - Assembles the complete toolkit (search + MCP + reflection)
- **get_today_str** - Provides current date context for research
- Token limit handling utilities for graceful degradation

These are defined in [`open_deep_library/utils.py`](open_deep_library/utils.py)

In [3]:
# Import utility functions and tools from the library
from open_deep_library.utils import (
    # Search tool - Lines 43-136: Tavily search with automatic summarization
    tavily_search,
    
    # Reflection tool - Lines 219-244: Strategic thinking tool for ReAct pattern
    think_tool,
    
    # Tool assembly - Lines 569-597: Get all configured tools
    get_all_tools,
    
    # Date utility - Lines 872-879: Get formatted current date
    get_today_str,
    
    # Supporting utilities for error handling
    get_api_key_for_model,          # Lines 892-914: Get API keys from config or env
    is_token_limit_exceeded,         # Lines 665-701: Detect token limit errors
    get_model_token_limit,           # Lines 831-846: Look up model's token limit
    remove_up_to_last_ai_message,    # Lines 848-866: Truncate messages for retry
    anthropic_websearch_called,      # Lines 607-637: Detect Anthropic native search usage
    openai_websearch_called,         # Lines 639-658: Detect OpenAI native search usage
    get_notes_from_tool_calls,       # Lines 599-601: Extract notes from tool messages
)

## Task 4: Configuration System

The configuration system controls:

### Research Behavior
- **allow_clarification** - Whether to ask clarifying questions before research
- **max_concurrent_research_units** - How many parallel researchers can run (default: 5)
- **max_researcher_iterations** - How many times supervisor can delegate research (default: 6)
- **max_react_tool_calls** - Tool call limit per researcher (default: 10)

### Model Configuration
- **research_model** - Model for research and supervision (we'll use Anthropic)
- **compression_model** - Model for synthesizing findings
- **final_report_model** - Model for writing the final report
- **summarization_model** - Model for summarizing web search results

### Search Configuration
- **search_api** - Which search API to use (ANTHROPIC, TAVILY, or NONE)
- **max_content_length** - Character limit before summarization

Defined in [`open_deep_library/configuration.py`](open_deep_library/configuration.py)

In [4]:
# Import configuration from the library
from open_deep_library.configuration import (
    Configuration,    # Lines 38-247: Main configuration class with all settings
    SearchAPI,        # Lines 11-17: Enum for search API options (ANTHROPIC, TAVILY, NONE)
)

## Task 5: Prompt Templates

The system uses carefully engineered prompts for each phase:

### Phase 1: Clarification
**clarify_with_user_instructions** - Analyzes if the research scope is clear or needs clarification

### Phase 2: Research Brief
**transform_messages_into_research_topic_prompt** - Converts user messages into a detailed research brief

### Phase 3: Supervisor
**lead_researcher_prompt** - System prompt for the supervisor that manages delegation strategy

### Phase 4: Researcher
**research_system_prompt** - System prompt for individual researchers conducting focused research

### Phase 5: Compression
**compress_research_system_prompt** - Prompt for synthesizing research findings without losing information

### Phase 6: Final Report
**final_report_generation_prompt** - Comprehensive prompt for writing the final report

All prompts are defined in [`open_deep_library/prompts.py`](open_deep_library/prompts.py)

In [5]:
# Import prompt templates from the library
from open_deep_library.prompts import (
    clarify_with_user_instructions,                    # Lines 3-41: Ask clarifying questions
    transform_messages_into_research_topic_prompt,     # Lines 44-77: Generate research brief
    lead_researcher_prompt,                            # Lines 79-136: Supervisor system prompt
    research_system_prompt,                            # Lines 138-183: Researcher system prompt
    compress_research_system_prompt,                   # Lines 186-222: Research compression prompt
    final_report_generation_prompt,                    # Lines 228-308: Final report generation
)

## ❓ Question #1:

Explain the interrelationships between the three states (Agent, Supervisor, Researcher). Why don't we just make a single huge state?

##### Answer: The hierarchical three state architecture is a deliberate design pattern that solves several critical problems in multi-agent systems. 
1. Each state level has its own responsibilities: Agent manages the overall conversation context, final report and user-facing information; Supervisor handles coordination logic and research iterations; Researcher is specialized for gathering context using tools and findings.

If everything lived in one huge state, we'd have chaotic mixing of user messages and internal researcher tool outputs, all of the outputs polluting final reports, and individual researchers iterations visible to all other researchers.

Other problem includes context bloat where each researcher would be doing max 5 iterations furthermore confusing the LLM with irrelevant context.


## ❓ Question #2:

What are the advantages and disadvantages of importing these components instead of including them in the notebook?


##### Answer: 

Advantage: shared components across multiple notebooks/projects, better development experience - everything we need to change or use is in one place, organized; readability and most important abstraction.

Disadvantages: cannot share just the notebook with someone, context switching, relying on dependecies import, path and environment management, worse debugging experience - when changing something we need to refresh imported files.


## 🏗️ Activity #1: Explore the Prompts

Open `open_deep_library/prompts.py` and examine one of the prompt templates in detail.

**Requirements:**
1. Choose one prompt template (clarify, brief, supervisor, researcher, compression, or final report)
2. Explain what the prompt is designed to accomplish
3. Identify 2-3 key techniques used in the prompt (e.g., structured output, role definition, examples)
4. Suggest one improvement you might make to the prompt

**YOUR CODE HERE** - Write your analysis in a markdown cell below

1. research_system_prompt
2. this prompt is designed to create a bounded, strategic research agent that mimics human research behavior.

3. 
- a) "You are a research assistant conducting research on the user's input topic. For context, today's date is {date}." - role clarity, he knows his job is research assistant (not supervisor or expert) 
- b) "<Hard Limits>
    **Tool Call Budgets** (Prevent excessive searching):
    - **Simple queries**: Use 2-3 search tool calls maximum
    - **Complex queries**: Use up to 5 search tool calls maximum
    - **Always stop**: After 5 search tool calls if you cannot find the right sources" - limiting researcher to cut costs and not get stuck in a loop.
- c) "<Show Your Thinking>
    After each search tool call, use think_tool to analyze the results:
    - What key information did I find?
    - What's missing?
    - Do I have enough to answer the question comprehensively?
    - Should I search more or provide my answer?
    </Show Your Thinking>" - forcing the agent to evaluate its own progress and discourages him from blindly calling multiple searches in parallel without assessment. (We should learn from this to sometimes stop and process what we just read or saw :)
    
4. Early exit heuristics with examples
Current problem is that agents overshoot because they're unsure if they've met the threshold.

Proposed addition to the stop/exit criteria:
"""
Example 1 - Simple Query:
Question: "What is the capital of France?"
STOP after 1 search: Found "Paris" from reliable source (eg. Wikipedia)
DON'T search for: "history of Paris", "Paris population", "Paris landmarks"

Example 2 - Complex Query:
Question: "What are evidence-based stress management techniques?"
STOP after 3 searches when you have:
- 5+ distinct techniques with explanations
- Scientific backing (studies, expert sources)
- Practical implementation steps
DON'T search for: Every possible technique, minute details on neuroscience
"""

---

# 🤝 Breakout Room #2
## Building & Running the Researcher

In this breakout room, we'll explore the node functions, build the graph, and run wellness research.

## Task 6: Node Functions - The Building Blocks

Now let's look at the node functions that make up our graph. We'll import them from the library and understand what each does.

### The Complete Research Workflow

The workflow consists of 8 key nodes organized into 3 subgraphs:

1. **Main Graph Nodes:**
   - `clarify_with_user` - Entry point that checks if clarification is needed
   - `write_research_brief` - Transforms user input into structured research brief
   - `final_report_generation` - Synthesizes all research into final report

2. **Supervisor Subgraph Nodes:**
   - `supervisor` - Lead researcher that plans and delegates
   - `supervisor_tools` - Executes supervisor's tool calls (delegation, reflection)

3. **Researcher Subgraph Nodes:**
   - `researcher` - Individual researcher conducting focused research
   - `researcher_tools` - Executes researcher's tool calls (search, reflection)
   - `compress_research` - Synthesizes researcher's findings

All nodes are defined in [`open_deep_library/deep_researcher.py`](open_deep_library/deep_researcher.py)

### Node 1: clarify_with_user

**Purpose:** Analyzes user messages and asks clarifying questions if the research scope is unclear.

**Key Steps:**
1. Check if clarification is enabled in configuration
2. Use structured output to analyze if clarification is needed
3. If needed, end with a clarifying question for the user
4. If not needed, proceed to research brief with verification message

**Implementation:** [`open_deep_library/deep_researcher.py` lines 60-115](open_deep_library/deep_researcher.py#L60-L115)

In [6]:
# Import the clarify_with_user node
from open_deep_library.deep_researcher import clarify_with_user

### Node 2: write_research_brief

**Purpose:** Transforms user messages into a structured research brief for the supervisor.

**Key Steps:**
1. Use structured output to generate detailed research brief from messages
2. Initialize supervisor with system prompt and research brief
3. Set up supervisor messages with proper context

**Why this matters:** A well-structured research brief helps the supervisor make better delegation decisions.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 118-175](open_deep_library/deep_researcher.py#L118-L175)

In [7]:
# Import the write_research_brief node
from open_deep_library.deep_researcher import write_research_brief

### Node 3: supervisor

**Purpose:** Lead research supervisor that plans research strategy and delegates to sub-researchers.

**Key Steps:**
1. Configure model with three tools:
   - `ConductResearch` - Delegate research to a sub-agent
   - `ResearchComplete` - Signal that research is done
   - `think_tool` - Strategic reflection before decisions
2. Generate response based on current context
3. Increment research iteration count
4. Proceed to tool execution

**Decision Making:** The supervisor uses `think_tool` to reflect before delegating research, ensuring thoughtful decomposition of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 178-223](open_deep_library/deep_researcher.py#L178-L223)

In [8]:
# Import the supervisor node (from supervisor subgraph)
from open_deep_library.deep_researcher import supervisor

### Node 4: supervisor_tools

**Purpose:** Executes the supervisor's tool calls, including strategic thinking and research delegation.

**Key Steps:**
1. Check exit conditions:
   - Exceeded maximum iterations
   - No tool calls made
   - `ResearchComplete` called
2. Process `think_tool` calls for strategic reflection
3. Execute `ConductResearch` calls in parallel:
   - Spawn researcher subgraphs for each delegation
   - Limit to `max_concurrent_research_units` (default: 5)
   - Gather all results asynchronously
4. Aggregate findings and return to supervisor

**Parallel Execution:** This is where the magic happens - multiple researchers work simultaneously on different aspects of the research question.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 225-349](open_deep_library/deep_researcher.py#L225-L349)

In [9]:
# Import the supervisor_tools node
from open_deep_library.deep_researcher import supervisor_tools

### Node 5: researcher

**Purpose:** Individual researcher that conducts focused research on a specific topic.

**Key Steps:**
1. Load all available tools (search, MCP, reflection)
2. Configure model with tools and researcher system prompt
3. Generate response with tool calls
4. Increment tool call iteration count

**ReAct Pattern:** Researchers use `think_tool` to reflect after each search, deciding whether to continue or provide their answer.

**Available Tools:**
- Search tools (Tavily or Anthropic native search)
- `think_tool` for strategic reflection
- `ResearchComplete` to signal completion
- MCP tools (if configured)

**Implementation:** [`open_deep_library/deep_researcher.py` lines 365-424](open_deep_library/deep_researcher.py#L365-L424)

In [10]:
# Import the researcher node (from researcher subgraph)
from open_deep_library.deep_researcher import researcher

### Node 6: researcher_tools

**Purpose:** Executes the researcher's tool calls, including searches and strategic reflection.

**Key Steps:**
1. Check early exit conditions (no tool calls, native search used)
2. Execute all tool calls in parallel:
   - Search tools fetch and summarize web content
   - `think_tool` records strategic reflections
   - MCP tools execute external integrations
3. Check late exit conditions:
   - Exceeded `max_react_tool_calls` (default: 10)
   - `ResearchComplete` called
4. Continue research loop or proceed to compression

**Error Handling:** Safely handles tool execution errors and continues with available results.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 435-509](open_deep_library/deep_researcher.py#L435-L509)

In [11]:
# Import the researcher_tools node
from open_deep_library.deep_researcher import researcher_tools

### Node 7: compress_research

**Purpose:** Compresses and synthesizes research findings into a concise, structured summary.

**Key Steps:**
1. Configure compression model
2. Add compression instruction to messages
3. Attempt compression with retry logic:
   - If token limit exceeded, remove older messages
   - Retry up to 3 times
4. Extract raw notes from tool and AI messages
5. Return compressed research and raw notes

**Why Compression?** Researchers may accumulate lots of tool outputs and reflections. Compression ensures:
- All important information is preserved
- Redundant information is deduplicated
- Content stays within token limits for the final report

**Token Limit Handling:** Gracefully handles token limit errors by progressively truncating messages.

**Implementation:** [`open_deep_library/deep_researcher.py` lines 511-585](open_deep_library/deep_researcher.py#L511-L585)

In [12]:
# Import the compress_research node
from open_deep_library.deep_researcher import compress_research

### Node 8: final_report_generation

**Purpose:** Generates the final comprehensive research report from all collected findings.

**Key Steps:**
1. Extract all notes from completed research
2. Configure final report model
3. Attempt report generation with retry logic:
   - If token limit exceeded, truncate findings by 10%
   - Retry up to 3 times
4. Return final report or error message

**Token Limit Strategy:**
- First retry: Use model's token limit × 4 as character limit
- Subsequent retries: Reduce by 10% each time
- Graceful degradation with helpful error messages

**Report Quality:** The prompt guides the model to create well-structured reports with:
- Proper headings and sections
- Inline citations
- Comprehensive coverage of all findings
- Sources section at the end

**Implementation:** [`open_deep_library/deep_researcher.py` lines 607-697](open_deep_library/deep_researcher.py#L607-L697)

In [13]:
# Import the final_report_generation node
from open_deep_library.deep_researcher import final_report_generation

## Task 7: Graph Construction - Putting It All Together

The system is organized into three interconnected graphs:

### 1. Researcher Subgraph (Bottom Level)
Handles individual focused research on a specific topic:
```
START → researcher → researcher_tools → compress_research → END
               ↑            ↓
               └────────────┘ (loops until max iterations or ResearchComplete)
```

### 2. Supervisor Subgraph (Middle Level)
Manages research delegation and coordination:
```
START → supervisor → supervisor_tools → END
            ↑              ↓
            └──────────────┘ (loops until max iterations or ResearchComplete)
            
supervisor_tools spawns multiple researcher_subgraphs in parallel
```

### 3. Main Deep Researcher Graph (Top Level)
Orchestrates the complete research workflow:
```
START → clarify_with_user → write_research_brief → research_supervisor → final_report_generation → END
                 ↓                                       (supervisor_subgraph)
               (may end early if clarification needed)
```

Let's import the compiled graphs from the library.

In [14]:
# Import the pre-compiled graphs from the library
from open_deep_library.deep_researcher import (
    # Bottom level: Individual researcher workflow
    researcher_subgraph,    # Lines 588-605: researcher → researcher_tools → compress_research
    
    # Middle level: Supervisor coordination
    supervisor_subgraph,    # Lines 351-363: supervisor → supervisor_tools (spawns researchers)
    
    # Top level: Complete research workflow
    deep_researcher,        # Lines 699-719: Main graph with all phases
)

## Why This Architecture?

### Advantages of Supervisor-Researcher Delegation

1. **Dynamic Task Decomposition**
   - Unlike section-based approaches with predefined structure, the supervisor can break down research based on the actual question
   - Adapts to different types of research (comparisons, lists, deep dives, etc.)

2. **Parallel Execution**
   - Multiple researchers work simultaneously on different aspects
   - Much faster than sequential section processing
   - Configurable parallelism (1-20 concurrent researchers)

3. **ReAct Pattern for Quality**
   - Researchers use `think_tool` to reflect after each search
   - Prevents excessive searching and improves search quality
   - Natural stopping conditions based on information sufficiency

4. **Flexible Tool Integration**
   - Easy to add MCP tools for specialized research
   - Supports multiple search APIs (Anthropic, Tavily)
   - Each researcher can use different tool combinations

5. **Graceful Token Limit Handling**
   - Compression prevents token overflow
   - Progressive truncation in final report generation
   - Research can scale to arbitrary depths

### Trade-offs

- **Complexity:** More moving parts than section-based approach
- **Cost:** Parallel researchers use more tokens (but faster)
- **Unpredictability:** Research structure emerges dynamically

## Task 8: Running the Deep Researcher

Now let's see the system in action! We'll use it to research wellness strategies for improving sleep quality.

### Setup

We need to:
1. Set up the wellness research request
2. Configure the execution with Anthropic settings
3. Run the research workflow

In [15]:
# Set up the graph with Anthropic configuration
from IPython.display import Markdown, display
import uuid

# Note: deep_researcher is already compiled from the library
# For this demo, we'll use it directly without additional checkpointing
graph = deep_researcher

print("✓ Graph ready for execution")
print("  (Note: The graph is pre-compiled from the library)")

✓ Graph ready for execution
  (Note: The graph is pre-compiled from the library)


### Configuration for Anthropic

We'll configure the system to use:
- **Claude Sonnet 4** for all research, supervision, and report generation
- **Tavily** for web search (you can also use Anthropic's native search)
- **Moderate parallelism** (1 concurrent researcher for cost control)
- **Clarification enabled** (will ask if research scope is unclear)

In [17]:
# Configure for Anthropic with moderate settings
config = {
    "configurable": {
        # Model configuration - using Claude Sonnet 4 for everything
        "research_model": "openai:gpt-5",
        "research_model_max_tokens": 10000,
        
        "compression_model": "openai:gpt-5",
        "compression_model_max_tokens": 8192,
        
        "final_report_model": "openai:gpt-5",
        "final_report_model_max_tokens": 10000,
        
        "summarization_model": "openai:gpt-5",
        "summarization_model_max_tokens": 8192,
        
        # Research behavior
        "allow_clarification": True,
        "max_concurrent_research_units": 1,  # 1 parallel researcher
        "max_researcher_iterations": 2,      # Supervisor can delegate up to 2 times
        "max_react_tool_calls": 3,           # Each researcher can make up to 3 tool calls
        
        # Search configuration
        "search_api": "tavily",  # Using Tavily for web search
        "max_content_length": 50000,
        
        # Thread ID for this conversation
        "thread_id": str(uuid.uuid4())
    }
}

print("✓ Configuration ready")
print(f"  - Research Model: GPT 5")
print(f"  - Max Concurrent Researchers: 1")
print(f"  - Max Iterations: 2")
print(f"  - Search API: Tavily")

✓ Configuration ready
  - Research Model: GPT 5
  - Max Concurrent Researchers: 1
  - Max Iterations: 2
  - Search API: Tavily


### Execute the Wellness Research

Now let's run the research! We'll ask the system to research evidence-based strategies for improving sleep quality.

The workflow will:
1. **Clarify** - Check if the request is clear (may skip if obvious)
2. **Research Brief** - Transform our request into a structured brief
3. **Supervisor** - Plan research strategy and delegate to researchers
4. **Parallel Research** - Researchers gather information simultaneously
5. **Compression** - Each researcher synthesizes their findings
6. **Final Report** - All findings combined into comprehensive report

In [17]:
# Create our wellness research request
research_request = """
I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please research the best evidence-based strategies for improving sleep quality and create a comprehensive sleep improvement plan for me.
"""

# Execute the graph
async def run_research():
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run the research
await run_research()

Starting research workflow...


Node: clarify_with_user

I have sufficient information to proceed with your sleep improvement research request. I understand that you're looking for evidence-based strategies to address your current sleep challenges, which include inconsistent bedtimes (10pm-1am), phone use in bed, and morning fatigue. I will now research the most effective, scientifically-backed sleep hygiene practices and create a comprehensive, personalized sleep improvement plan that addresses your specific issues.

Node: write_research_brief

Research Brief Generated:
I want to improve my sleep quality by developing a comprehensive, evidence-based sleep improvement plan. My current sleep challenges include: going to bed at inconsistent times (ranging from 10pm to 1am), using my phone in bed, and often feeling tired in the morning despite getting sleep. Please research the most effective, scientifically-backed sleep hygiene strategies and interventions that specifically address incon


Node: research_supervisor

Node: final_report_generation

FINAL REPORT GENERATED



Error generating final report: Error code: 429 - {'type': 'error', 'error': {'type': 'rate_limit_error', 'message': "This request would exceed your organization's rate limit of 30,000 input tokens per minute (org: 36a0367d-2b4e-43a2-aa03-a9c1a1c5ba6c, model: claude-sonnet-4-20250514). For details, refer to: https://docs.claude.com/en/api/rate-limits. You can see the response headers for current usage. Please reduce the prompt length or the maximum tokens requested, or try again later. You may also contact sales at https://www.anthropic.com/contact-sales to discuss your options for a rate limit increase."}, 'request_id': 'req_011CXqUs7cwdNcqPBQvQLVjS'}


Research workflow completed!


## Task 9: Understanding the Output

Let's break down what happened:

### Phase 1: Clarification
The system checked if your request was clear. Since you provided specific details about your sleep issues, it likely proceeded without asking clarifying questions.

### Phase 2: Research Brief
Your request was transformed into a detailed research brief that guides the supervisor's delegation strategy.

### Phase 3: Supervisor Delegation
The supervisor analyzed the brief and decided how to break down the research:
- Used `think_tool` to plan strategy
- Called `ConductResearch` to delegate to researchers
- Each delegation specified a focused research topic (e.g., sleep hygiene, circadian rhythm, blue light effects)

### Phase 4: Parallel Research
Researchers worked on their assigned topics:
- Each researcher used web search tools to gather information
- Used `think_tool` to reflect after each search
- Decided when they had enough information
- Compressed their findings into clean summaries

### Phase 5: Final Report
All research findings were synthesized into a comprehensive sleep improvement plan with:
- Well-structured sections
- Evidence-based recommendations
- Practical action items
- Sources for further reading

## Task 10: Key Takeaways & Next Steps

### Architecture Benefits
1. **Dynamic Decomposition** - Research structure emerges from the question, not predefined
2. **Parallel Efficiency** - Multiple researchers work simultaneously
3. **ReAct Quality** - Strategic reflection improves search decisions
4. **Scalability** - Handles token limits gracefully through compression
5. **Flexibility** - Easy to add new tools and capabilities

### When to Use This Pattern
- **Complex research questions** that need multi-angle investigation
- **Comparison tasks** where parallel research on different topics is beneficial
- **Open-ended exploration** where structure should emerge dynamically
- **Time-sensitive research** where parallel execution speeds up results

### When to Use Section-Based Instead
- **Highly structured reports** with predefined format requirements
- **Template-based content** where sections are always the same
- **Sequential dependencies** where later sections depend on earlier ones
- **Budget constraints** where token efficiency is critical

### Extend the System
1. **Add MCP Tools** - Integrate specialized tools for your domain
2. **Custom Prompts** - Modify prompts for specific research types
3. **Different Models** - Try different Claude versions or mix models
4. **Persistence** - Use a real database for checkpointing instead of memory

### Learn More
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [Open Deep Research Repo](https://github.com/langchain-ai/open_deep_research)
- [Anthropic Claude Documentation](https://docs.anthropic.com/)
- [Tavily Search API](https://tavily.com/)

## ❓ Question #3:

What are the trade-offs of using parallel researchers vs. sequential research? When might you choose one approach over the other?

#### Answer:

#### Parallel researchers
- Pros: faster, independent exploration, fault-tolerant, better coverage
- Cons: higher API costs, duplicate searches, coordination overhead, no knowledge sharing

#### Sequential Research
- Pros: lower cost, builds on findings, logical flow of events, handles dependencies
- Cons: slower, bias accumulation, single point of failure, less diverse

#### Decision:
- Parallel: on independent questions, where speed matters, such as "compare X vs Y" tasks
- Sequential: on logical dependencies, where budget matters, and narrative flow is needed

## ❓ Question #4:

How would you adapt this deep research architecture for a production wellness application? What additional components would you need?

##### Answer:
- Horizontal scaling for multiple users
- Docker containerization for consistent deployment across environments
- User authentication
- Replace in-memory stores with persistent storage
- Logging (eg. built-in LangGraph)
Extra:
- Human-in-the-loop to flag uncertain recommendations for clinician review or reach out to depressive users.

## 🏗️ Activity #2: Custom Wellness Research

Using what you've learned, run a custom wellness research task.

**Requirements:**
1. Create a wellness-related research question (exercise, nutrition, stress, etc.)
2. Modify the configuration for your use case
3. Run the research and analyze the output
4. Document what worked well and what could be improved

**Experiment ideas:**
- Research exercise routines for specific conditions (bad knee, lower back pain)
- Compare different stress management techniques
- Investigate nutrition strategies for specific goals
- Explore meditation and mindfulness research

**YOUR CODE HERE**

In [18]:
# YOUR CODE HERE
# Create your own wellness research request and run it

my_wellness_request = """
I want to know what I can eat. I have:
- autoimune Hashimoto's
- prediabetic state

Please research the best evidence-based nutrition strategies and create a comprehensive meal plan for me.
"""

# Optionally modify the config
my_config = {
    "configurable": {
        "research_model": "openai:gpt-5",
        "research_model_max_tokens": 10000,
        "compression_model": "openai:gpt-5",
        "compression_model_max_tokens": 8192,
        "final_report_model": "openai:gpt-5",
        "final_report_model_max_tokens": 10000,
        "summarization_model": "openai:gpt-5",
        "summarization_model_max_tokens": 8192,
        "allow_clarification": True,
        "max_concurrent_research_units": 1,
        "max_researcher_iterations": 2,
        "max_react_tool_calls": 3,
        "search_api": "tavily",
        "max_content_length": 50000,
        "thread_id": str(uuid.uuid4())
    }
}

async def run_custom_research(research_request, config):
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run your research
await run_custom_research(my_wellness_request, my_config)

Starting research workflow...


Node: clarify_with_user

To tailor an evidence-based plan for Hashimoto’s and prediabetes, could you share:

- Age, sex, height, current weight, typical weekly activity level, and primary goal (lose/maintain/gain; target rate if losing)
- Other health conditions or life stages (e.g., kidney disease, hypertension, high cholesterol, pregnancy/breastfeeding)
- Meds/supplements: Do you take levothyroxine? What time? Any diabetes meds (e.g., metformin/GLP‑1), iron or calcium supplements?
- Diet restrictions/preferences: allergies or intolerances; vegetarian/vegan; gluten-free or diagnosed celiac; lactose intolerance; religious restrictions; any foods you avoid (e.g., soy)
- Food likes/dislikes and preferred cuisines; budget and cooking time/skill; prefer quick meals or batch-cooking
- Meal structure: meals/snacks per day; desired plan length (e.g., 7-day) and number of servings (just you or household)
- Recent numbers (if known): A1C or fasting glucose

Resea

In [20]:
# YOUR CODE HERE
# Create your own wellness research request and run it

my_wellness_request = """
I want to know what I can eat. I have:
- autoimune Hashimoto's
- prediabetic state
- Female, age: 30 years, 161cm, 68kgs
- weekly activity: gym 2-3 times and walking
- no meds
- Diet: based on Hashimoto, it is prefered gluten-free
- Gluten-free: preference/flexible
- Primary goal: weight loss
- dietary restrictions: none
- eat animal products
- food preferance: asian cuisine


Please research the best evidence-based nutrition strategies and create a comprehensive meal plan for me.
"""

# Optionally modify the config
my_config = {
    "configurable": {
        "research_model": "openai:gpt-5",
        "research_model_max_tokens": 10000,
        "compression_model": "openai:gpt-5",
        "compression_model_max_tokens": 8192,
        "final_report_model": "openai:gpt-5",
        "final_report_model_max_tokens": 10000,
        "summarization_model": "openai:gpt-5",
        "summarization_model_max_tokens": 8192,
        "allow_clarification": True,
        "max_concurrent_research_units": 1,
        "max_researcher_iterations": 2,
        "max_react_tool_calls": 3,
        "search_api": "tavily",
        "max_content_length": 50000,
        "thread_id": str(uuid.uuid4())
    }
}

async def run_custom_research(research_request, config):
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)

# Run your research
await run_custom_research(my_wellness_request, my_config)

Starting research workflow...


Node: clarify_with_user

Thanks—this is sufficient to proceed. I understand you’re a 30-year-old female (161 cm, 68 kg), active (gym 2–3x/week plus walking), with Hashimoto’s thyroiditis and prediabetes, not on medication, omnivorous with a flexible gluten-free preference, primary goal of weight loss, and a preference for Asian cuisine with no other dietary restrictions. I’ll now begin evidence-based research and create tailored nutrition strategies and a comprehensive meal plan for you.

Node: write_research_brief

Research Brief Generated:
What are the best evidence-based nutrition strategies and a detailed, Asian-cuisine–forward, mostly gluten‑free (preference, flexible) omnivorous meal plan for me—a 30‑year‑old female (161 cm, 68 kg; BMI ≈26.2), physically active (gym 2–3x/week plus walking), with autoimmune Hashimoto’s thyroiditis and prediabetes, on no medications—whose primary goal is weight loss? Please: 

1) Determine energy needs and targets
- 

Error generating final report: Connection error.


Research workflow completed!


In [21]:
# YOUR CODE HERE
# Create your own wellness research request and run it

my_wellness_request = """
I want to know what I can eat. I have:
- autoimune Hashimoto's
- prediabetic state
- Female, age: 30 years, 161cm, 68kgs
- weekly activity: gym 2-3 times and walking
- no meds
- Diet: based on Hashimoto, it is prefered gluten-free
- Gluten-free: preference/flexible
- Primary goal: weight loss
- dietary restrictions: none
- eat animal products
- food preferance: asian cuisine


Please research the best evidence-based nutrition strategies and create a comprehensive meal plan for me.
"""

# Optionally modify the config
my_config = {
    "configurable": {
        "research_model": "openai:gpt-5",
        "research_model_max_tokens": 10000,
        "compression_model": "openai:gpt-5",
        "compression_model_max_tokens": 8192,
        "final_report_model": "openai:gpt-4o",  # ← Try more reliable model
        "final_report_model_max_tokens": 8000,  # ← Reduce tokens to speed up
        "summarization_model": "openai:gpt-5",
        "summarization_model_max_tokens": 8192,
        "allow_clarification": True,
        "max_concurrent_research_units": 1,
        "max_researcher_iterations": 2,
        "max_react_tool_calls": 3,
        "search_api": "tavily",
        "max_content_length": 30000,  # ← Reduce to prevent timeout
        "thread_id": str(uuid.uuid4()),
        # Add these:
        "request_timeout": 120,  # 2 minutes
        "max_retries": 3
    }
}


async def run_custom_research(research_request, config):
    """Run the research workflow and display results."""
    print("Starting research workflow...\n")
    
    async for event in graph.astream(
        {"messages": [{"role": "user", "content": research_request}]},
        config,
        stream_mode="updates"
    ):
        final_state = event
        # Display each step
        for node_name, node_output in event.items():
            print(f"\n{'='*60}")
            print(f"Node: {node_name}")
            print(f"{'='*60}")
            
            if node_name == "clarify_with_user":
                if "messages" in node_output:
                    last_msg = node_output["messages"][-1]
                    print(f"\n{last_msg.content}")
            
            elif node_name == "write_research_brief":
                if "research_brief" in node_output:
                    print(f"\nResearch Brief Generated:")
                    print(f"{node_output['research_brief'][:500]}...")
            
            elif node_name == "supervisor":
                print(f"\nSupervisor planning research strategy...")
                if "supervisor_messages" in node_output:
                    last_msg = node_output["supervisor_messages"][-1]
                    if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
                        print(f"Tool calls: {len(last_msg.tool_calls)}")
                        for tc in last_msg.tool_calls:
                            print(f"  - {tc['name']}")
            
            elif node_name == "supervisor_tools":
                print(f"\nExecuting supervisor's tool calls...")
                if "notes" in node_output:
                    print(f"Research notes collected: {len(node_output['notes'])}")
            
            elif node_name == "final_report_generation":
                if "final_report" in node_output:
                    print(f"\n" + "="*60)
                    print("FINAL REPORT GENERATED")
                    print("="*60 + "\n")
                    display(Markdown(node_output["final_report"]))
                else:
                    # ← Add error handling here
                    print("⚠️ Final report generation had an issue")
                    if "notes" in node_output:
                        print(f"✅ Research completed: {len(node_output.get('notes', ''))} chars")
    
    print("\n" + "="*60)
    print("Research workflow completed!")
    print("="*60)
    
    return final_state  # ← Return the state

# Run your research
final_state = await run_custom_research(my_wellness_request, my_config)

Starting research workflow...


Node: clarify_with_user

Thank you—this is sufficient to proceed. I understand you’re a 30-year-old female (161 cm, 68 kg) with autoimmune Hashimoto’s (no meds) and prediabetes, aiming for weight loss. You’re active (gym 2–3x/week + walking), prefer Asian cuisine, eat animal products, and are flexible with gluten-free. I will now begin evidence-based research and prepare a comprehensive nutrition strategy and meal plan aligned with your goals and preferences.

Node: write_research_brief

Research Brief Generated:
What are the most robust, up-to-date (through 2026) evidence-based nutrition strategies and a practical, culturally aligned meal plan for me—a 30-year-old female, 161 cm, 68 kg (BMI ~26), with autoimmune Hashimoto’s thyroiditis (not on thyroid medication) and prediabetes—whose primary goal is weight loss; who exercises at the gym 2–3 times per week plus walking; eats animal products; has no dietary restrictions; prefers Asian cuisine; and prefer

# Comprehensive Nutrition Strategy and Meal Plan for Weight Loss and Glycemic Control

## Introduction

For a 30-year-old female with autoimmune Hashimoto’s thyroiditis and prediabetes, effective management of weight and glycemic control through diet is crucial. This report provides a comprehensive nutrition strategy, focusing on evidence-based dietary patterns, macronutrient distribution, and meal timing to support her weight loss goals. The preferences for Asian cuisine and flexibility with gluten-free eating will be integrated into a practical, culturally aligning meal plan.

## Evidence Synthesis and Rationale

### Dietary Patterns and Weight/Glycemic Outcomes in Prediabetes

- **Calorie-Restricted Diets**: Evidence supports that a calorie deficit of approximately 20% or about 500 kcal/day is effective for weight loss, improving insulin sensitivity and reducing A1c levels.
- **Mediterranean Diet**: Rich in healthy fats (MUFA/PUFA), this diet improves glycemic control and cardiovascular health. It's linked with improved A1c and lipid profiles and can be adapted to include Asian flavors.
- **Low-Carb/Low-GI Diets**: These diets are beneficial in lowering fasting glucose and A1c. Prioritizing low-GI Asian staples such as basmati rice over high-GI jasmine rice can be effective.
- **High-Protein Diets**: Consuming 1.2–1.6 g/kg/day, with protein evenly distributed across meals, supports satiety and lean mass retention during weight loss phases.

### Carbohydrate Quality and Distribution

- **Daily Carbohydrate Intake**: Aim for 130–150 g of carbohydrates per day, distributed evenly (30–45 g per meal) to maintain stable blood glucose levels.
- **Glycemic Index/Load**: Choose low to medium-GI foods. Asian staples like soba noodles or brown rice are preferable to jasmine rice and instant noodles.
- **Fiber Intake**: Target 25–35 g/day to aid in glycemic control and weight management. Incorporate fiber-rich legumes and vegetables.
- **Added Sugar Limit**: Keep to less than 25 g/day.

### Protein Needs for Weight Loss and Glycemic Control

- **Protein Target**: 1.2–1.6 g/kg/day, promoting muscle retention. Include animal-based (fish, chicken) and plant-based proteins (tofu, legumes).
- **Protein Distribution**: Approximately 20–35 g per meal to enhance satiety.

### Fat Quality and Sodium Considerations

- **Fat Quality**: Increase MUFAs from sources like sesame oil and PUFAs from fatty fish. Include omega-3-rich foods at least twice a week.
- **Sodium**: Limit to 1500–2300 mg/day, especially considering high-sodium Asian condiments. Use low-sodium soy sauce or tamari.

### Hashimoto’s-Specific Considerations

- **Gluten-Free Diet**: While there’s no conclusive evidence for gluten-free benefits in non-celiac Hashimoto’s patients, many choose to eliminate gluten to potentially reduce thyroid antibodies. Supplement with nutrient-rich gluten-free grains to avoid deficiencies.
- **Micronutrients**: Balance the intake of iodine, selenium, zinc, iron, vitamin D, and B12, essential for thyroid health.
  - **Iodine**: Avoid excess. Do not exceed 150 mcg/day without medical supervision.
  - **Selenium**: 55 mcg/day from sources like Brazil nuts and seafood.
  - **Zinc**: 8 mg/day from meat and seeds.
  - **Vitamin D**: Consider supplementation if dietary sunlight exposure is inadequate.
- **Goitrogens and Soy**: Cooking reduces goitrogenic effects. Soy can be included unless further adverse effects are noted.

### Meal Timing Strategies

- **Time-Restricted Eating (TRE)**: Adopting early eating windows (8 am – 4 pm) may enhance weight loss and glycemic control, according to recent trials.

## Individualization and Targets

### Energy Needs and Caloric Deficit

- **Energy Needs**: Using the Mifflin-St Jeor equation, estimated daily caloric needs are about 1900 kcal/day, assuming moderate activity. Aiming for 1400–1500 kcal/day can yield effective weight loss.

### Macronutrient Distribution

- **Carbohydrates**: 35-40% of total intake; focus on low-GI, high-fiber foods.
- **Proteins**: 25-30% to support muscle mass retention.
- **Fats**: 30-35% with an emphasis on healthy fats.

## Practical Outputs

### 7-Day Meal Plan

Create a 7-day meal plan comprising gluten-free alternatives with Asian cuisine influences and dairy options. Include:

- **Breakfasts**: Incorporate high-protein, low-GI ingredients (e.g., egg and tofu stir fry).
- **Lunches/Dinners**: Balance lean proteins, vegetables, and low-GI carbohydrates (e.g., soba noodles with chicken and vegetables).
- **Snacks**: Opt for nuts, mixed seeds, and yogurt.

### Grocery List and Meal Prep

Compile a list of core ingredients for the meal plan and a preparation guide focusing on batch cooking and easy-to-make meals.

### Dining Out Tips

- Choose dishes centered around lean proteins and vegetables.
- Request modifications like steamed rice instead of fried rice.
- Use gluten-free tamari in place of regular soy sauce when available.

### Monitoring and Adjustments

- Track weight, waist circumference, and glucose levels regularly.
- Schedule periodic follow-ups with healthcare providers to monitor thyroid function and adjust dietary micronutrient intake accordingly.

## Sources

[1] ADA Standards of Care: URL  
[2] American Thyroid Association Guidelines: URL  
[3] Cochrane Review on Diet in Prediabetes: URL  
[4] University of Sydney Glycemic Index Database: URL  
[5] NIH Office of Dietary Supplements Fact Sheets: URL


Research workflow completed!
